<a href="https://colab.research.google.com/github/mickymultani/RAG-Testing-Gemma7B-IT/blob/main/RAG_Gemma7B_IT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U -q "transformers==4.38.1" --upgrade

In [ ]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q llama-index
!pip install -q gradio
!pip install -q einops
!pip install -q accelerate
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-embeddings-fastembed
!pip install -q fastembed

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

documents = SimpleDirectoryReader("/content/Data").load_data()

In [ ]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 512

In [ ]:
from llama_index.core import PromptTemplate

system_prompt = "You are a NASA Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."

query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 1.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="google/gemma-7b-it",
    model_name="google/gemma-7b-it",
    device_map="cuda",
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}
)

Settings.llm = llm
Settings.chunk_size = 512

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

def predict(input, history):
  response = query_engine.query(input)
  return str(response)

In [ ]:
import time
import gradio as gr

def predict(input, history):
    start_time = time.time()  # Start the timer

    response = query_engine.query(input)  # Process the query

    end_time = time.time()  # Stop the timer
    response_time = end_time - start_time  # Calculate the time taken

    # Format the response to include the time taken
    timed_response = f"{response}\n\n(Response Time: {response_time:.2f} seconds)"
    return str(timed_response)

# Launch gradio chat ui
gr.ChatInterface(predict).launch(share=True, debug=True)